<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Google GenAI SDK - Building an Orchestrator Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai arize-phoenix-otel openinference-instrumentation-google-genai

## Connect to Arize Phoenix

In [ ]:
import os
from getpass import getpass

from google import genai
from google.genai import types

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(auto_instrument=True, project_name="google-genai-orchestrator-agent")
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="<ADD YOUR GCP PROJECT ID>", location="us-central1")

# Orchestration Agent

First, define the sub agents, or in this case tools, that the orchestrator can choose between.

In [ ]:
# Define models for different specialized agents
FLASH_MODEL = "gemini-2.0-flash-001"


@tracer.chain()
def call_user_proxy_agent(query, context=""):
    """User proxy agent that acts as the user and gives feedback."""
    prompt = f"""You are a user proxy assistant. Provide feedback as if you were the user on:
    Context: {context}
    Query: {query}
    Give honest, constructive feedback from a user's perspective."""

    response = client.models.generate_content(
        model=FLASH_MODEL,
        contents=prompt,
    )
    return response.text.strip()


@tracer.chain()
def call_flight_planning_agent(query, context=""):
    """Flight planning agent that helps find and recommend flights."""
    prompt = f"""You are a flight planning assistant. Help plan flights for:
    Context: {context}
    Query: {query}
    Provide detailed flight options with considerations for price, timing, and convenience."""

    response = client.models.generate_content(
        model=FLASH_MODEL,
        contents=prompt,
    )
    return response.text.strip()


@tracer.chain()
def call_hotel_recommendation_agent(query, context=""):
    """Hotel recommendation agent that suggests accommodations."""
    prompt = f"""You are a hotel recommendation assistant. Suggest accommodations for:
    Context: {context}
    Query: {query}
    Provide suitable hotel options with details on amenities, location, and price ranges."""

    response = client.models.generate_content(
        model=FLASH_MODEL,
        contents=prompt,
    )
    return response.text.strip()


@tracer.chain()
def call_travel_attraction_agent(query, context=""):
    """Travel attraction recommendation agent that suggests places to visit."""
    prompt = f"""You are a travel attraction recommendation assistant. Suggest attractions for:
    Context: {context}
    Query: {query}
    Provide interesting places to visit with descriptions, highlights, and practical information."""

    response = client.models.generate_content(
        model=FLASH_MODEL,
        contents=prompt,
    )
    return response.text.strip()

In [ ]:
@tracer.chain()
def determine_next_step(user_query, context, cycle, max_cycles):
    """
    Determines the next agent to call based on the current context and user query.
    Args:
        user_query: The initial user query
        context: Current accumulated context
        cycle: Current cycle number
        max_cycles: Maximum number of agent calls
    Returns:
        The function name to call next
    """
    orchestration_prompt = f"""You are an orchestration agent. Decide the next step to take:
    User query: {user_query}
    Current context: {context}
    Current cycle: {cycle}/{max_cycles}
    Choose one of the available tools to help address the user query, or decide to return a final answer.
    """

    # Define orchestrator tools
    orchestrator_tools = {
        "function_declarations": [
            {
                "name": "call_planning_agent",
                "description": "Call planning agent to create a structured plan with next steps",
            },
            {
                "name": "call_flight_planning_agent",
                "description": "Call flight planning agent to help find and recommend flights",
            },
            {
                "name": "call_hotel_recommendation_agent",
                "description": "Call hotel recommendation agent to suggest accommodations",
            },
            {
                "name": "call_travel_attraction_agent",
                "description": "Call travel attraction agent to suggest interesting places to visit with descriptions",
            },
            {
                "name": "call_user_proxy_agent",
                "description": "Call user proxy agent that acts as the user and gives feedback",
            },
            {
                "name": "return_final_answer",
                "description": "Return to user with final answer when sufficient information has been gathered",
            },
        ]
    }

    orchestration_response = client.models.generate_content(
        model=FLASH_MODEL,
        contents=orchestration_prompt,
        config=types.GenerateContentConfig(tools=[orchestrator_tools]),
    )

    if orchestration_response.candidates[0].content.parts[0].function_call:
        function_call = orchestration_response.candidates[0].content.parts[0].function_call
        return function_call.name
    else:
        return "return_final_answer"  # Default to returning final answer if no tool called


@tracer.chain()
def execute_agent_call(function_name, user_query, context):
    """
    Executes the specified agent call and returns the response and agent type.
    Args:
        function_name: The name of the function to call
        user_query: The initial user query
        context: Current accumulated context
    Returns:
        Tuple of (agent_response, agent_type)
    """
    if function_name == "call_flight_planning_agent":
        agent_response = call_flight_planning_agent(user_query, context)
        agent_type = "Flight Planning"
    elif function_name == "call_hotel_recommendation_agent":
        agent_response = call_hotel_recommendation_agent(user_query, context)
        agent_type = "Hotel Recommendation"
    elif function_name == "call_travel_attraction_agent":
        agent_response = call_travel_attraction_agent(user_query, context)
        agent_type = "Travel Attraction"
    elif function_name == "call_user_proxy_agent":
        agent_response = call_user_proxy_agent(user_query, context)
        agent_type = "User Proxy"
    else:
        agent_response = ""
        agent_type = "Unknown"

    return agent_response, agent_type


@tracer.chain()
def generate_final_answer(user_query, context, max_cycles_reached=False):
    """
    Generates a final answer based on the accumulated context.
    Args:
        user_query: The initial user query
        context: Current accumulated context
        max_cycles_reached: Whether the maximum cycles were reached
    Returns:
        Final response to the user
    """
    final_prompt = f"""Create a final response to the user query: {user_query}
    Based on this context: {context}
    """

    if max_cycles_reached:
        final_prompt += "\n\nProvide a comprehensive and helpful answer, noting that we've reached our maximum processing cycles."
    else:
        final_prompt += "\n\nProvide a comprehensive and helpful answer."

    final_response = client.models.generate_content(
        model=FLASH_MODEL,
        contents=final_prompt,
    )

    return final_response.text.strip()


@tracer.agent()
def orchestrator(user_query, max_cycles=3):
    """
    Orchestrator that decides which agent to call at each step of the process.
    Args:
        user_query: The initial user query
        max_cycles: Maximum number of agent calls before returning to user
    Returns:
        Final response to the user
    """
    context = ""
    cycle = 0

    while cycle < max_cycles:
        # Determine next step
        function_name = determine_next_step(user_query, context, cycle, max_cycles)

        if function_name == "return_final_answer":
            return generate_final_answer(user_query, context)

        # Execute the agent call
        agent_response, agent_type = execute_agent_call(function_name, user_query, context)

        # Update context with agent response
        context += f"\n\n{agent_type} Agent Output:\n{agent_response}"
        cycle += 1

    # If max cycles reached, return what we have
    return generate_final_answer(user_query, context, max_cycles_reached=True)


# Example usage
user_query = """I want to plan a 5-day trip to Paris, France, sometime in October. I'm interested
in museums and good food. Find flight options from SFO, suggest mid-range hotels near the city center,
and recommend some relevant activities."""

response = orchestrator(user_query)
print(response)